# Direction Finding

The direction of the sound source is here estimated by determining for the same signal the relative delays between the different hydrophones and by considering the following relation between measured delays and the direction vector of the sound source. 

The minimal number of hydrophones to form a volumetric array is 4 allowing to form n = 6 different hydrophone pairs.
To obtain an optimal volumetric array the hydrophones are located on the tips of an tetrahedron. This way all 6 hydrophone pairs have the same separation.
While direction finding could be done with only 3 hydrophone pairs, the use of all 6 different combinations should improve the performance, as the impact of measurement errors is reduced. 
This is reflected by the Least-Mean-Square notation in equation. 
Also, using all possible hydrophone pairs puts the reference location of the array automatically in the center of the array eliminating the need for a selection of a reference hydrophone.

A volumetric array with 6 hydrophones allows the formation of n = 15 different pairs of hydrophones. If the hydrophones are located on the tips of an octohedron, 12 of the 15 hydrophone pairs have the same length, and 3 are slightly longer.

## Basic formulation
Let the sound direction  $\hat s$ be given by 
\begin{equation}
    \hat s = 
    \left(
        \begin{array}{c}
            s_x\\
            s_y\\
            s_z
        \end{array}
    \right)  
\end{equation}
and the hydrophone separation matrix $ D_0$ be given by

\begin{equation}
     D_0 = 
    \left(
        \begin{array}{c c c c}
            d_{1x} & d_{2x} & ... & d_{nx}\\
            d_{1y} & d_{2y} & ... & d_{ny}\\
            d_{1z} & d_{2z} & ... & d_{nz}\\
        \end{array}
    \right)  
\end{equation}
where the elements $(d_{ix},d_{iy},d_{iz}) $ are the $x,y,z$ components of the vector that connects a pair of hydrophones.

then the vector of measurements $ m$ defined by 
\begin{equation}
     m = 
    \left(
        \begin{array}{c}
            m_1\\
            m_2\\
            \vdots\\
            m_n
        \end{array}
    \right)  
\end{equation}

is estimated by the projection of the sound source direction onto the different hydrophone pairs.


\begin{equation}
     m =  D_0^T  {\hat s}
\end{equation}

The sound direction is then estimated from the measurement vector by

\begin{equation}
    {\hat s} = \left( D_0^T  D_0 \right)^{-1}  D_0  m
\end{equation}


## Angle estimations

The arrival angles azimuth $\alpha$ and elevation $\beta$  of the sound are finally estimated by solving

\begin{equation}
    \tan \alpha = \frac{-s_y}{s_x}
\end{equation}

and

\begin{equation}
    \sin \beta = s_z
\end{equation}

or alternatively

\begin{equation}
    \tan \beta = \frac{s_z}{\sqrt{s_x^2+s_y^2}}
\end{equation}

As the azimuth angles may vary from -180°  to 180°  the solutions require the usual care for  $s_x<0$, i.e. by using the four-quadrant inverse tangent. 
The minus sign in equation is simply due to the convention that positive azimuth is measured towards starboard. 

As the elevation angle $\beta$ is restricted to ±90°, an ordinary inverse sinus is sufficient but a four-quadrant inverse tangent seems also here preferred because it is insensitive to a common factor (e.g. uncertain sound speed at the array).

## Implementation
The following function demonstrates the generation of the matrix $DI = \left( D_0^T  D_0 \right)^{-1}  D_0$ that is multiplied with the measurement vector to obtain the sound direction. 

The two hydrophone arrays are two systems that have been used (cPAM) or are in use by the author (cVAS).


In [6]:
import numpy as np

def hydrophones(model='cVAS)'):
    if model=='cVAS':
        # cVAS hydrophones
        # octahedron (x points from 'centre' towards hydrophone 1)
        ro=0.040 # m
        dz=np.sqrt(2)
        ang=np.array([0,120,240,60,180,300])*np.pi/180
        #
        ho=np.empty((len(ang),3))
        ho[:,:]=np.transpose([np.cos(ang[:]),np.sin(ang[:]),-dz/2+0*ang[:]])
        ho[3:,2] += dz
        ho *= ro
    if model=='cPAM':
        # cPAM hydrophones
        # tripple towed array (x points forward, 'towards hydrophone 1)
        ro=0.56;
        dz=1.0;
        ang=np.array([-120,-120,120,120,0,0])*np.pi/180.0;
        #
        ho=np.empty((len(ang),3));
        ho[:,:]=np.transpose([dz/2+0*ang[:],ro*np.sin(ang[:]),ro*np.cos(ang[:])]);
        ho[range(1,6,2),0] -= dz;        
    #
    isel=np.array([[0,1],[0,2],[0,3],[0,4],[0,5],
                [1,2],[1,3],[1,4],[1,5],
                [2,3],[2,4],[2,5],
                [3,4],[3,5],
                [4,5]])

    [nc,_]=np.shape(isel)
    #print(nc)

    D=ho[isel[:,1],]-ho[isel[:,0],]
    L=np.sqrt(np.sum(D**2,1))
    #print(L)
    DI=np.linalg.pinv(D)
    
    return isel,DI,nc


## Time delay based direction finding

For a time delay based direction finding one uses as measurements
\begin{equation}
     m_i = c \Delta \tau_i
\end{equation}
where $c$ is the sound speed between the a pair of hydrophones,

## Intensity based diection finding

Using the estimated (spectral) sound intensity along a pair of hydrophones, say $I_i(f)$ then suitable measurements are

\begin{equation}
m_i = \Delta x_{12}I_i(f)    =-\frac{1}{\rho} \frac{\text{Im}\{P_{i2}(\omega) P_{i1}^* (\omega)\}}{\omega}
\end{equation}
where $P_{i1}(\omega)$ and $P_{i2}(\omega)$ are the (complex valued) power spectra of a pair of hydrophones and $\Delta x_{12}$ id the distance between the two hydrophones. 

Intensity based direction finding is spectral based and therefore limited to lower frequencies $k\omega < 2/d$, where $d$ is the maximal hydrophone spacing